In [55]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
!pip install kagglehub
import kagglehub
rounakbanik_the_movies_dataset_path = kagglehub.dataset_download('rounakbanik/the-movies-dataset')

print('Data source import complete.')


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Data source import complete.


# Data load, clean and format

In [56]:
!pip install numpy pandas matplotlib
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import os
import time
import ast
import timeit

!pip install langid
import langid

!pip install transformers torch
from transformers import pipeline

pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 100)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [57]:
movies_df = pd.read_csv('client/movies_metadata.csv', low_memory=False)
ratings_df = pd.read_csv('client/ratings_small.csv')
credits_df = pd.read_csv('client/credits.csv')

In [58]:
movies_df.shape

(45466, 24)

In [59]:
ratings_df.shape

(100006, 4)

In [60]:
credits_df.shape

(45476, 3)

In [61]:
movies_df.dtypes

adult                     object
belongs_to_collection     object
budget                    object
genres                    object
homepage                  object
id                        object
imdb_id                   object
original_language         object
original_title            object
overview                  object
popularity                object
poster_path               object
production_companies      object
production_countries      object
release_date              object
revenue                  float64
runtime                  float64
spoken_languages          object
status                    object
tagline                   object
title                     object
video                     object
vote_average             float64
vote_count               float64
dtype: object

In [62]:
ratings_df.dtypes

userId         int64
movieId        int64
rating       float64
timestamp      int64
dtype: object

In [63]:
credits_df.dtypes

cast    object
crew    object
id       int64
dtype: object

In [64]:
movies_df.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', 'poster_path': '/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg',...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, {'id': 10751, 'name': 'Family'}]",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his room until Andy's birthday brings Buzz Lightyear o...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States of America'}]",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, 'name': 'Fantasy'}, {'id': 10751, 'name': 'Family'}]",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an enchanted board game that opens the door to a magical w...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'name': 'Teitler Film', 'id': 2550}, {'name': 'Inters...","[{'iso_3166_1': 'US', 'name': 'United States of America'}]",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso_639_1': 'fr', 'name': 'Français'}]",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collection', 'poster_path': '/nLvUdqgPgm3F85NMCii9gVFUcet...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, 'name': 'Comedy'}]",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud between next-door neighbors and fishing buddies John...,11.7129,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,"[{'name': 'Warner Bros.', 'id': 6194}, {'name': 'Lancaster Gate', 'id': 19464}]","[{'iso_3166_1': 'US', 'name': 'United States of America'}]",1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for Love.,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'name': 'Drama'}, {'id': 10749, 'name': 'Romance'}]",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the women are holding their breath, waiting for the elusi...",3.859495,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,"[{'name': 'Twentieth Century Fox Film Corporation', 'id': 306}]","[{'iso_3166_1': 'US', 'name': 'United States of America'}]",1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself... and never let you forget it.,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Collection', 'poster_path': '/nts4iOmNnq7GNicycMJ9pSA...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,"Just when George Banks has recovered from his daughter's wedding, he receives the news that she'...",8.387519,/e64sOI48hQXyru7naBFyssKFxVd.jpg,"[{'name': 'Sandollar Productions', 'id': 5842}, {'name': 'Touchstone Pictures', 'id': 9195}]","[{'iso_3166_1': 'US', 'name': 'United States of America'}]",1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's In For The Surprise Of His Life!,Father of the Bride Part II,False,5.7,173.0


In [65]:
ratings_df.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [66]:
credits_df.head()

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)', 'credit_id': '52fe4284c3a36847f8024f95', 'gender'...","[{'credit_id': '52fe4284c3a36847f8024f49', 'department': 'Directing', 'gender': 2, 'id': 7879, '...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', 'credit_id': '52fe44bfc3a36847f80a7c73', 'gender': ...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'department': 'Production', 'gender': 2, 'id': 511, '...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'credit_id': '52fe466a9251416c75077a8d', 'gender': 2...","[{'credit_id': '52fe466a9251416c75077a89', 'department': 'Directing', 'gender': 2, 'id': 26502, ...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah' Jackson"", 'credit_id': '52fe44779251416c91011aad...","[{'credit_id': '52fe44779251416c91011acb', 'department': 'Directing', 'gender': 2, 'id': 2178, '...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', 'credit_id': '52fe44959251416c75039eb9', 'gender': ...","[{'credit_id': '52fe44959251416c75039ed7', 'department': 'Sound', 'gender': 2, 'id': 37, 'job': ...",11862


In [67]:
movies_df["id"].nunique()

45436

In [68]:
ratings_df["movieId"].nunique()

9068

In [69]:
credits_df["id"].nunique()

45432

## Dataframes Cleaning

In [70]:
movies_df_columns = ["id", "title", "genres", "release_date", "overview", "production_countries", "spoken_languages", "budget", "revenue"]
ratings_df_columns = ["movieId", "rating", "timestamp"]
credits_df_columns = ["id", "cast"]

In [71]:
# Discard unuseful columns
movies_df_cleaned = movies_df.dropna(subset=movies_df_columns)[movies_df_columns].copy()
ratings_df_cleaned = ratings_df.dropna(subset=ratings_df_columns)[ratings_df_columns].copy()
credits_df_cleaned = credits_df.dropna(subset=credits_df_columns)[credits_df_columns].copy()

In [72]:
# Dates Formatting
movies_df_cleaned['release_date'] = pd.to_datetime(movies_df_cleaned['release_date'], format='%Y-%m-%d', errors='coerce')
ratings_df_cleaned['timestamp'] = pd.to_datetime(ratings_df_cleaned['timestamp'], unit='s')

In [73]:
# Numbers Formatting
movies_df_cleaned['budget'] = pd.to_numeric(movies_df_cleaned['budget'], errors='coerce')
movies_df_cleaned['revenue'] = pd.to_numeric(movies_df_cleaned['revenue'], errors='coerce')

In [74]:
# Replace json fields with string arrays
def dictionary_to_list(dictionary_str):
    try:
        dictionary_list = ast.literal_eval(dictionary_str)
        return [data['name'] for data in dictionary_list]
    except (ValueError, SyntaxError):
        return []

movies_df_cleaned['genres'] = movies_df_cleaned['genres'].apply(dictionary_to_list)
movies_df_cleaned['production_countries'] = movies_df_cleaned['production_countries'].apply(dictionary_to_list)
movies_df_cleaned['spoken_languages'] = movies_df_cleaned['spoken_languages'].apply(dictionary_to_list)
credits_df_cleaned['cast'] = credits_df_cleaned['cast'].apply(dictionary_to_list)

In [75]:
movies_df_cleaned['genres'] = movies_df_cleaned['genres'].astype(str)
movies_df_cleaned['production_countries'] = movies_df_cleaned['production_countries'].astype(str)
movies_df_cleaned['spoken_languages'] = movies_df_cleaned['spoken_languages'].astype(str)

In [76]:
movies_df_cleaned.head()

,id,title,genres,release_date,overview,production_countries,spoken_languages,budget,revenue
0,862,Toy Story,"['Animation', 'Comedy', 'Family']",1995-10-30,"Led by Woody, Andy's toys live happily in his room until Andy's birthday brings Buzz Lightyear o...",['United States of America'],['English'],30000000,373554033.0
1,8844,Jumanji,"['Adventure', 'Fantasy', 'Family']",1995-12-15,When siblings Judy and Peter discover an enchanted board game that opens the door to a magical w...,['United States of America'],"['English', 'Français']",65000000,262797249.0
2,15602,Grumpier Old Men,"['Romance', 'Comedy']",1995-12-22,A family wedding reignites the ancient feud between next-door neighbors and fishing buddies John...,['United States of America'],['English'],0,0.0
3,31357,Waiting to Exhale,"['Comedy', 'Drama', 'Romance']",1995-12-22,"Cheated on, mistreated and stepped on, the women are holding their breath, waiting for the elusi...",['United States of America'],['English'],16000000,81452156.0
4,11862,Father of the Bride Part II,['Comedy'],1995-02-10,"Just when George Banks has recovered from his daughter's wedding, he receives the news that she'...",['United States of America'],['English'],0,76578911.0


In [77]:
ratings_df_cleaned.head()

,movieId,rating,timestamp
0,31,2.5,2009-12-14 02:52:24
1,1029,3.0,2009-12-14 02:52:59
2,1061,3.0,2009-12-14 02:53:02
3,1129,2.0,2009-12-14 02:53:05
4,1172,4.0,2009-12-14 02:53:25


In [78]:
credits_df_cleaned.head()

,id,cast
0,862,"[Tom Hanks, Tim Allen, Don Rickles, Jim Varney, Wallace Shawn, John Ratzenberger, Annie Potts, J..."
1,8844,"[Robin Williams, Jonathan Hyde, Kirsten Dunst, Bradley Pierce, Bonnie Hunt, Bebe Neuwirth, David..."
2,15602,"[Walter Matthau, Jack Lemmon, Ann-Margret, Sophia Loren, Daryl Hannah, Burgess Meredith, Kevin P..."
3,31357,"[Whitney Houston, Angela Bassett, Loretta Devine, Lela Rochon, Gregory Hines, Dennis Haysbert, M..."
4,11862,"[Steve Martin, Diane Keaton, Martin Short, Kimberly Williams-Paisley, George Newbern, Kieran Cul..."


In [79]:
movies_df_cleaned.dtypes

id                              object
title                           object
genres                          object
release_date            datetime64[ns]
overview                        object
production_countries            object
spoken_languages                object
budget                           int64
revenue                        float64
dtype: object

In [80]:
ratings_df_cleaned.dtypes

movieId               int64
rating              float64
timestamp    datetime64[ns]
dtype: object

In [81]:
credits_df_cleaned.dtypes

id       int64
cast    object
dtype: object

# Queries a Resolver

- Q1: Peliculas y sus géneros de los años 2000 con producción Argentina y Española.
- Q2: Top 5 de países que más dinero han invertido en producciones sin colaborar con otros países.
- Q3: Películas de Producción Argentina estrenadas a partir del 2000, con mayor y con menor promedio de rating.
- Q4: Top 10 de actores con mayor participación en películas de producción Argentina con
fecha de estreno posterior al 2000
- Q5: Average de la tasa ingreso/presupuesto de peliculas con overview de sentimiento positivo vs. sentimiento negativo, para películas de habla inglesa con producción americana, estrenadas a partir del año 2000

### Q1:  Peliculas y sus géneros de los años 00' con producción Argentina y Española Q2:

In [82]:
movies_argentina_españa_00s_df = movies_df_cleaned[
    (movies_df_cleaned['production_countries'].str.contains('Argentina', case=False, na=False)) &
    (movies_df_cleaned['production_countries'].str.contains('Spain', case=False, na=False)) &
    (movies_df_cleaned['release_date'].dt.year >= 2000) &
    (movies_df_cleaned['release_date'].dt.year < 2010)
]

In [83]:
movies_argentina_españa_00s_df.shape


(24, 9)

In [84]:
movies_argentina_españa_00s_df[["title", "genres"]]

,title,genres
4695,La Cienaga,"['Comedy', 'Drama']"
4739,Burnt Money,['Crime']
7970,The City of No Limits,"['Thriller', 'Drama']"
9385,Nicotina,"['Drama', 'Action', 'Comedy', 'Thriller']"
9437,Lost Embrace,"['Drama', 'Foreign']"
9953,Whisky,"['Comedy', 'Drama', 'Foreign']"
10325,The Holy Girl,"['Drama', 'Foreign']"
10638,The Aura,"['Crime', 'Drama', 'Thriller']"
10739,Bombón: The Dog,['Drama']
11504,Rolling Family,"['Drama', 'Comedy']"


### Q2: Top 5 de países que más dinero han invertido en producciones sin colaborar con otros países.

In [85]:
solo_country_df = movies_df_cleaned[movies_df_cleaned['production_countries'].apply(lambda x: len(eval(x)) == 1)]

In [86]:
solo_country_df.loc[:, 'country'] = solo_country_df['production_countries'].apply(lambda x: eval(x)[0])

/var/folders/73/87cl_fwd24z_3gj382d3c4mc0000gn/T/ipykernel_13145/2161155210.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  solo_country_df.loc[:, 'country'] = solo_country_df['production_countries'].apply(lambda x: eval(x)[0])


In [87]:
investment_by_country = solo_country_df.groupby('country')['budget'].sum().sort_values(ascending=False)

In [88]:
top_5_countries = investment_by_country.head(5)

print(top_5_countries)

country
United States of America    120153886644
France                        2256831838
United Kingdom                1611604610
India                         1169682797
Japan                          832585873
Name: budget, dtype: int64


### Q3: Películas de Producción Argentina estrenadas a partir del 2000, con mayor y con menor promedio de rating.

In [89]:
movies_argentina_post_2000_df = movies_df_cleaned[
    (movies_df_cleaned['production_countries'].str.contains('Argentina', case=False, na=False)) &
    (movies_df_cleaned['release_date'].dt.year >= 2000)
]

In [90]:
movies_argentina_post_2000_df = movies_argentina_post_2000_df.astype({'id': int})

In [91]:
ranking_arg_post_2000_df = movies_argentina_post_2000_df[["id", "title"]].merge(ratings_df_cleaned,
                                                                                left_on="id",
                                                                                right_on="movieId")
mean_ranking_arg_post_2000_df = ranking_arg_post_2000_df.groupby(["id", "title"])['rating'].mean().reset_index()

In [92]:
#Max
mean_ranking_arg_post_2000_df.iloc[mean_ranking_arg_post_2000_df['rating'].idxmax()]

id                         125619
title     The forbidden education
rating                        4.0
Name: 9, dtype: object

In [93]:
#Min
mean_ranking_arg_post_2000_df.iloc[mean_ranking_arg_post_2000_df['rating'].idxmin()]

id               128598
title     Left for Dead
rating              1.0
Name: 10, dtype: object

### Q4: Top 10 de actores con mayor participación en películas de producción Argentina posterior al 2000

In [94]:
cast_arg_post_2000_df = movies_argentina_post_2000_df[["id", "title"]].merge(credits_df_cleaned,
                                                                                on="id")
cast_and_movie_arg_post_2000_df = cast_arg_post_2000_df.set_index("id")["cast"].apply(pd.Series).stack().reset_index("id", name="name")
cast_per_movie_quantities = cast_and_movie_arg_post_2000_df.groupby(["name"]).count().reset_index().rename(columns={"id":"count"})
cast_per_movie_quantities.nlargest(10, 'count')

,name,count
1010,Ricardo Darín,17
47,Alejandro Awada,7
489,Inés Efron,7
664,Leonardo Sbaraglia,7
1125,Valeria Bertuccelli,7
111,Arturo Goetz,6
285,Diego Peretti,6
948,Pablo Echarri,6
998,Rafael Spregelburd,6
1033,Rodrigo de la Serna,6


### Q5: Average de la tasa ingreso/presupuesto de peliculas con overview de sentimiento positivo vs. sentimiento negativo

In [95]:
q5_input_df = movies_df_cleaned.copy()

In [96]:
# Nos quedamos con rows válidas
q5_input_df = q5_input_df.loc[q5_input_df['budget'] != 0]
q5_input_df = q5_input_df.loc[q5_input_df['revenue'] != 0]

In [97]:
q5_input_df.shape

(5370, 9)

In [98]:
# Cargar modelo preentrenado para análisis de sentimiento
sentiment_analyzer = pipeline('sentiment-analysis', model='distilbert-base-uncased-finetuned-sst-2-english')

Device set to use mps:0


In [99]:
start_time = time.time()
q5_input_df['sentiment'] = q5_input_df['overview'].fillna('').apply(lambda x: sentiment_analyzer(x, truncation=True)[0]['label'])
elapsed_time = time.time() - start_time
print(f"Execution time: {elapsed_time:.2f} seconds")

Execution time: 51.91 seconds


In [100]:
q5_input_df["rate_revenue_budget"] = q5_input_df["revenue"] / q5_input_df["budget"]

In [101]:
q5_input_df.sample(10)

,id,title,genres,release_date,overview,production_countries,spoken_languages,budget,revenue,sentiment,rate_revenue_budget
899,705,All About Eve,['Drama'],1950-11-09,"From the moment she glimpses her idol at the stage door, Eve Harrington is determined to take th...",['United States of America'],"['English', 'Français']",1400000,63463.0,POSITIVE,0.045331
38718,290098,The Handmaiden,"['Thriller', 'Drama', 'Romance']",2016-06-01,"1930s Korea, in the period of Japanese occupation, a new girl (Sookee) is hired as a handmaiden ...",['South Korea'],"['日本語', '한국어/조선말']",8575000,1983204.0,POSITIVE,0.231277
21099,96936,The Bling Ring,"['Drama', 'Crime']",2013-06-12,"Inspired by actual events, a group of fame-obsessed teenagers use the Internet to track celebrit...",['United States of America'],['English'],15000000,19145732.0,NEGATIVE,1.276382
14778,12201,Edge of Darkness,"['Crime', 'Drama', 'Mystery', 'Thriller']",2010-01-29,"As a seasoned homicide detective, Thomas Craven has seen the bleakest side of humanity. But noth...","['United Kingdom', 'United States of America']",['English'],80000000,74901339.0,NEGATIVE,0.936267
9611,11633,Appleseed,"['Animation', 'Action', 'Science Fiction']",2004-04-17,"In a utopian society created at the end of the third world war, a female warrior who has been pl...",['Japan'],['日本語'],10000000,1461989.0,NEGATIVE,0.146199
3958,96,Beverly Hills Cop II,"['Action', 'Comedy', 'Crime']",1987-05-18,"Detroit cop, Axel Foley heads for the land of sunshine and palm trees to find out who shot polic...",['United States of America'],['English'],20000000,299965036.0,POSITIVE,14.998252
18795,94329,The Raid,"['Action', 'Thriller', 'Crime']",2011-09-08,Deep in the heart of Jakarta's slums lies an impenetrable safe house for the world's most danger...,"['Indonesia', 'United States of America', 'France']",['Bahasa indonesia'],1100000,4105187.0,NEGATIVE,3.731988
6389,302,Swimming Pool,"['Thriller', 'Crime']",2003-05-18,"In the middle of this amusing thriller is a relationship between two different types of females,...","['France', 'United Kingdom']","['English', 'Français']",7800000,22441323.0,POSITIVE,2.877093
12177,5123,August Rush,"['Family', 'Drama', 'Music']",2007-11-21,"A drama with fairy tale elements, where an orphaned musical prodigy uses his gift as a clue to f...",['United States of America'],['English'],25000000,66122026.0,POSITIVE,2.644881
23962,249164,If I Stay,['Drama'],2014-08-21,"Based on Gayle Forman's novel of the same name. ""If I Stay"" is the story of the gifted classical...",['United States of America'],['English'],11000000,78874843.0,POSITIVE,7.170440


In [102]:
average_rate_by_sentiment = q5_input_df.groupby("sentiment")["rate_revenue_budget"].mean()
print(average_rate_by_sentiment)

sentiment
NEGATIVE    5453.397595
POSITIVE    5668.650541
Name: rate_revenue_budget, dtype: float64
